In [25]:
import requests
import pandas as pd
from os import system
from coinbase.wallet.client import Client
from datetime import datetime as dt, timedelta

In [26]:
api_key = "your-api-key"
api_secret = "your-secret-key"
client = Client(api_key, api_secret)

In [27]:
def get_candles(sym, start, end):
    url = "https://api.exchange.coinbase.com"
    barSize = "86400"
    timeStart = start.isoformat()
    timeEnd = end.isoformat()
    parameters = {
        "start":timeStart,
        "end":timeEnd,
        "granularity":barSize
    }
    data = requests.get(f"{url}/products/{sym}/candles", 
                        params=parameters, 
                        headers= {"accept": "application/json"})
    return data

In [28]:
def kline_to_df(sym, start, end):
    kline_data  = eval(get_candles(sym, start, end).text)
    headers = ["timestamp", "low", "high", "open", "close", "volume"]
    df = pd.DataFrame(data=kline_data, columns=headers)
    df["timestamp"] = df["timestamp"].astype(int)
    for i in range(len(df["timestamp"])):
        df["timestamp"][i] = dt.fromtimestamp(df["timestamp"][i])
    df.iloc[:] = df.iloc[::-1].values
    return df

In [29]:
def year_df(sym, filename):
    df1 = kline_to_df(sym, dt(2022, 1, 2, 0, 0, 0, 0),dt(2022, 7, 1, 0, 0, 0, 0) )
    df2 = kline_to_df(sym, dt(2022, 7, 2, 0, 0, 0, 0),dt.now() )
    df = pd.concat([df1, df2], axis=0)
    df.to_csv(filename, index=False)
    return df

In [ ]:
year_df("LINK-USD", "coinbase_link_hist_data.csv")
year_df("ETH-USD", "coinbase_eth_hist_data.csv")
year_df("BTC-USD", "coinbase_btc_hist_data.csv")